<a href="https://colab.research.google.com/github/crychenet/analysis_of_financion_report/blob/master/ozonControlNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import sys
import datetime


initial_df = pd.read_excel('/content/Отчет по товарам за период 2023-08-01 - 2023-11-19.xlsx')

ouer_sales_report = pd.read_excel('/content/2023.10.31 Отчет по скидкам_021123.xlsx', skiprows=1)
ouer_stock = pd.read_excel("/content/Склад 20.xlsx")
ouer_price = pd.read_excel("/content/actual_price.xlsx")

ozon_prices = pd.read_excel('/content/Шаблон_цен OZON.xlsx', sheet_name='Товары и цены')
ozon_stock = pd.read_excel('/content/stock-report 21.11.xlsx')

days_on_sale = pd.read_excel('/content/Весь ассортимент продажи 2019-2023.xlsx', sheet_name='Лист2') # Наши дни в продаже

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [94]:
ozon_stock = pd.read_excel('/content/stock-report 21.11.xlsx')

In [95]:
initial_df['Сумма фактических продаж'] = initial_df['За продажу или возврат до вычета комиссий и услуг'] * initial_df['Количество']

sales_df = initial_df[initial_df['За продажу или возврат до вычета комиссий и услуг'] > 0][['Артикул', 'За продажу или возврат до вычета комиссий и услуг', 'Количество']]
return_df = initial_df[initial_df['За продажу или возврат до вычета комиссий и услуг'] < 0][['Артикул', 'За продажу или возврат до вычета комиссий и услуг', 'Количество']]

sales_df = sales_df.groupby('Артикул').sum()
sales_df['Сумма продаж'] = sales_df['За продажу или возврат до вычета комиссий и услуг']
sales_df['Кол-во продаж'] = sales_df['Количество']
sales_df.drop(columns=['За продажу или возврат до вычета комиссий и услуг', 'Количество'], inplace=True)
return_df = return_df.groupby('Артикул').sum()
return_df['Сумма возвратов'] = return_df['За продажу или возврат до вычета комиссий и услуг']
return_df['Кол-во возвратов'] = return_df['Количество']
return_df.drop(columns=['За продажу или возврат до вычета комиссий и услуг', 'Количество'], inplace=True)


merge_df = pd.merge(sales_df, return_df, how='outer', right_index=True, left_index=True)
merge_df.fillna(0, inplace=True)
merge_df['Итог выручки'] = merge_df['Сумма продаж'] + merge_df['Сумма возвратов']
merge_df['Итог продаж'] = merge_df['Кол-во продаж'] - merge_df['Кол-во возвратов']

merge_df.reset_index(inplace=True, drop=False)

In [96]:
test = pd.concat([pd.DataFrame({'Артикул': ozon_stock['Артикул'].unique()}),
                  pd.DataFrame({'Артикул': merge_df['Артикул'].unique()})])
test = pd.DataFrame({'Артикул': test['Артикул'].unique()})

In [97]:
merge_df = pd.merge(test, merge_df, on='Артикул', how='left')

In [98]:
merge_df = pd.merge(merge_df, initial_df[['Артикул', 'SKU']].groupby('Артикул').agg({'SKU': 'first'}).reset_index(), on='Артикул', how='left')
merge_df.rename(columns={'SKU': 'Артикул OZON'}, inplace=True)

merge_df.fillna(0, inplace=True)
vectorized_get_hyperlink_ozon = np.vectorize(lambda column: f"https://www.ozon.ru/product/{int(column)}")
merge_df['Ссылка OZON'] = vectorized_get_hyperlink_ozon(merge_df['Артикул OZON'])

In [99]:
def find_article(row):
    columns = row['Артикул']
    first_way = columns[:-3]
    second_way = columns[:-6]

    test = ouer_sales_report[ouer_sales_report['Артикул 1С'] == columns]
    if not test.empty:
        size = columns.replace(test['Артикул 1С'].iloc[0], '')
        if len(size) == 0:
            size = 0
        return pd.Series([test['Артикул 1С'].iloc[0], size], index=['Артикул 1С', 'Размер'])

    test = ouer_sales_report[ouer_sales_report['Артикул 1С'] == first_way]
    if not test.empty:
        size = columns.replace(test['Артикул 1С'].iloc[0], '')[1:]
        if len(size) == 0:
            size = 0
        return pd.Series([test['Артикул 1С'].iloc[0], size], index=['Артикул 1С', 'Размер'])

    test = ouer_sales_report[ouer_sales_report['Артикул 1С'] == second_way]
    if not test.empty:
        size = columns.replace(test['Артикул 1С'].iloc[0], '')[1:]
        if len(size) == 0:
            size = 0
        return pd.Series([test['Артикул 1С'].iloc[0], size], index=['Артикул 1С', 'Размер'])

# Apply the function row-wise using axis=1
merge_df[['Артикул 1С', 'Размер']] = merge_df.apply(find_article, axis=1)


In [43]:
# merge_df[merge_df['Артикул'].isin(['01MK-32-07A1GG-37', '01MK-32-07A1GG-39', '01MK-32-07A1GG-36'])]
test1 = return_df.reset_index()
test1[test1['Артикул'].isin(['01MK-32-07A1GG-37', '01MK-32-07A1GG-39', '01MK-32-07A1GG-36'])]

,Артикул,Сумма возвратов,Кол-во возвратов
8,01MK-32-07A1GG-37,-1570,1


In [44]:
test2 = sales_df.reset_index()
test2[test2['Артикул'].isin(['01MK-32-07A1GG-37', '01MK-32-07A1GG-39', '01MK-32-07A1GG-36'])]

,Артикул,Сумма продаж,Кол-во продаж
35,01MK-32-07A1GG-36,1494,1
36,01MK-32-07A1GG-37,5084,3


In [100]:
merge_df[merge_df['Артикул'].isin(['01MK-32-07A1GG-37', '01MK-32-07A1GG-39', '01MK-32-07A1GG-36'])]

,Артикул,Сумма продаж,Кол-во продаж,Сумма возвратов,Кол-во возвратов,Итог выручки,Итог продаж,Артикул OZON,Ссылка OZON,Артикул 1С,Размер
95,01MK-32-07A1GG-37,5084.0,3.0,-1570.0,1.0,3514.0,2.0,175654394.0,https://www.ozon.ru/product/175654394,01MK-32-07A1GG,37
96,01MK-32-07A1GG-39,0.0,0.0,0.0,0.0,0.0,0.0,175654397.0,https://www.ozon.ru/product/175654397,01MK-32-07A1GG,39
97,01MK-32-07A1GG-36,1494.0,1.0,0.0,0.0,1494.0,1.0,175654400.0,https://www.ozon.ru/product/175654400,01MK-32-07A1GG,36


In [101]:
# brands = ['4x4 shoes', 'NOBBARO', 'Makfine', 'MAKFLY', 'VALSER', 'Vigorous', 'Benta']
brands = ['4x4', 'MakFly', 'Nobbaro', 'VALSER', 'Replica', "El'Rosso", 'Vigorous', 'Benta', 'MakFine', 'MAKFLY', '4х4', 'Makfine'] # Каждый раз хорошо бы проверять так как в названиях ошибки
merge_df = pd.merge(merge_df, ozon_prices[['Артикул', 'Текущая цена (со скидкой), руб.']],
                    on='Артикул', how='left')


def find_brand(product_title, list_brends):
    pattern = re.compile(rf'\b(?:{"|".join(re.escape(brand) for brand in brands)})\b')
    match = pattern.search(product_title)
    try:
        brand_information = match.start(), re.findall(pattern, product_title)[0] if match else None
        return brand_information
    except AttributeError:
        pass
        # print('______________________________________')
        # print(product_title)
        # print("_______________________________________")
def extract_brand_and_article_name(row):
    brand_information = find_brand(row, brands)
    try:
        brand_and_article_name = pd.Series([brand_information[1], row[:(brand_information[0]-1)]], index=['Бренд', 'Наименование товара'])
        return brand_and_article_name
    except TypeError:
        brand_and_article_name = pd.Series(['ПРОВЕРИТЬ', 'ПРОВЕРИТЬ'], index=['Бренд', 'Наименование товара'])
        return brand_and_article_name


ozon_stock = ozon_stock[['Название товара', 'Валид', 'Артикул', 'SKU']].groupby('SKU').agg({'Валид': 'sum', 'Артикул': 'first', 'Название товара': 'first'})
ozon_stock[['Бренд', 'Наименование товара']] = ozon_stock['Название товара'].apply(extract_brand_and_article_name)
merge_df = pd.merge(merge_df, ozon_stock[['Бренд', 'Наименование товара', 'Артикул', 'Валид']],
                    on='Артикул', how='left')

merge_df.rename(columns={'Валид': 'Остатки OZON, шт', 'Текущая цена (со скидкой), руб.': 'Актуальная цена'}, inplace=True)

In [102]:
merge_df = merge_df[['Артикул 1С', 'Итог продаж', 'Итог выручки',
                     'Актуальная цена', 'Бренд', 'Наименование товара', 'Остатки OZON, шт',
                     'Ссылка OZON']].groupby(
                          'Артикул 1С').agg({'Итог продаж': 'sum', 'Итог выручки': 'sum', 'Актуальная цена': 'max',
                                 'Бренд': 'first', 'Наименование товара': 'first',
                                 'Остатки OZON, шт' : 'sum', 'Ссылка OZON': 'first'}).reset_index()
merge_df = merge_df.replace('None', 0)
merge_df = merge_df.fillna(0)


In [103]:
merge_df[merge_df['Артикул 1С'] == '01MK-32-07A1GG']

,Артикул 1С,Итог продаж,Итог выручки,Актуальная цена,Бренд,Наименование товара,"Остатки OZON, шт",Ссылка OZON
15,01MK-32-07A1GG,11.0,17401.0,1606.0,MakFine,Туфли,13.0,https://www.ozon.ru/product/175654394


In [104]:
test = merge_df[['Артикул 1С', 'Итог продаж', 'Итог выручки',
                     'Актуальная цена', 'Бренд', 'Наименование товара', 'Остатки OZON, шт',
                     'Ссылка OZON']].groupby(
                          'Артикул 1С').agg({'Итог продаж': 'sum', 'Итог выручки': 'sum', 'Актуальная цена': 'max',
                                 'Бренд': 'first', 'Наименование товара': 'first',
                                 'Остатки OZON, шт' : 'sum', 'Ссылка OZON': 'first'})

test.reset_index(inplace=True)
test[test['Артикул 1С'] == '01MK-32-07A1GG']

,Артикул 1С,Итог продаж,Итог выручки,Актуальная цена,Бренд,Наименование товара,"Остатки OZON, шт",Ссылка OZON
15,01MK-32-07A1GG,11.0,17401.0,1606.0,MakFine,Туфли,13.0,https://www.ozon.ru/product/175654394


In [105]:
def sort_warehouse(pandas_df):
    def calculate_sum(column):
        # Доделать исключения
        try:
            split_column = column.split(' ')
            sum_size_boxe = split_column[1]
            sum_size_boxe = re.sub(r'[(, )]', '', sum_size_boxe)
            split_column[0] = split_column[0].replace(';', ',')
            return split_column[0] + ' ' + str(sum([int(element) for element in list(sum_size_boxe)])) + ' ' + split_column[1]
        except:
            pass # Обратить внимание
    # ТИПА 23/24;25/26;27/28;29/30 (2222) не должно быть!
    pandas_df['Характеристика номенклатуры.Размеры'] = np.where(
        pandas_df['Характеристика номенклатуры.Размеры'].str.contains(';'),
        pandas_df['Характеристика номенклатуры.Размеры'].str.replace(';', ',').apply(calculate_sum),
        pandas_df['Характеристика номенклатуры.Размеры']
    )
    pandas_df['Характеристика номенклатуры.Размеры'] = pandas_df['Характеристика номенклатуры.Размеры'].astype(str)
    # Удалить строку Итог
    article_information = pandas_df['Характеристика номенклатуры.Размеры'].str.split(' ', expand=True)
    article_information.rename(columns={0: 'Размер', 1: 'Кол-во пар', 2: 'Список кол-ва размеров'}, inplace=True)
    # article_information.to_excel('article_information.xlsx')
    sorted_df = pd.DataFrame({
        'Артикул': pandas_df['Артикул'],
        'Размер': article_information['Размер'],
        'Кол-во пар': article_information['Кол-во пар'],
        'Список кол-ва размеров': article_information['Список кол-ва размеров'],
        'Свободно': pandas_df['Свободно'],
        'Способ доставки': np.where(
            (article_information['Список кол-ва размеров'].str.len() == 4) | (article_information['Список кол-ва размеров'].str.len() == 3),
            'Монокороб',
            np.where(
                article_information['Кол-во пар'].isnull(),
                'Штучно',
                'Ростовой короб'
            ))
    })
    return sorted_df


sorted_ouer_stock = sort_warehouse(ouer_stock)

In [52]:
# Следующая часть используется при разделении артикула по размерам
'''
multiplicity_rost_boxe = sorted_ouer_stock[sorted_ouer_stock['Способ доставки'] == 'Ростовой короб']

multiplicity_rost_boxe[['Начальный Размер', 'Конечный Размер']] = multiplicity_rost_boxe[~multiplicity_rost_boxe['Размер'].str.contains('/')]['Размер'].str.split('-', expand=True)
multiplicity_rost_boxe['Начальный Размер'] = pd.to_numeric(multiplicity_rost_boxe['Начальный Размер'])
multiplicity_rost_boxe['Конечный Размер'] = pd.to_numeric(multiplicity_rost_boxe['Конечный Размер'])

multiplicity_rost_boxe_first_part = multiplicity_rost_boxe.dropna(subset=['Начальный Размер', 'Конечный Размер'])

multiplicity_rost_boxe_first_part = multiplicity_rost_boxe_first_part.loc[multiplicity_rost_boxe_first_part.index.repeat(
    multiplicity_rost_boxe_first_part['Конечный Размер'] - multiplicity_rost_boxe_first_part['Начальный Размер'] + 1)].copy()
multiplicity_rost_boxe_first_part['Размер'] = multiplicity_rost_boxe_first_part.groupby(level=0).cumcount() + multiplicity_rost_boxe_first_part['Начальный Размер']
multiplicity_rost_boxe_first_part['Размер'] = multiplicity_rost_boxe_first_part['Размер'].astype(int)

multiplicity_rost_boxe_second_part = multiplicity_rost_boxe[multiplicity_rost_boxe['Размер'].str.contains('/')]

multiplicity_rost_boxe_second_part['Размер'] = multiplicity_rost_boxe_second_part['Размер'].str.split(',')
multiplicity_rost_boxe_second_part = multiplicity_rost_boxe_second_part.explode('Размер')

multiplicity_rost_boxe_second_part['Размер'] = multiplicity_rost_boxe_second_part['Размер'].str.replace('/', '-')

# Соединяем назад
multiplicity_rost_boxe = pd.concat([multiplicity_rost_boxe_first_part, multiplicity_rost_boxe_second_part])[['Артикул', 'Размер', 'Кол-во пар', 'Список кол-ва размеров', 'Свободно', 'Способ доставки']]

multiplicity_rost_boxe['Порядковый номер'] = multiplicity_rost_boxe.groupby(['Артикул', 'Список кол-ва размеров']).cumcount() + 1

list_position = []

for index, row in multiplicity_rost_boxe.iterrows():
    try:
        value = float(row['Список кол-ва размеров'][row['Порядковый номер']]) * (row['Свободно'] // float(row['Кол-во пар']))
        list_position.append(value)
    except:
        print(row['Артикул'])
multiplicity_rost_boxe['Кол-во пар для общего учета'] = list_position
'''

<ipython-input-52-c1458ae7d2b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiplicity_rost_boxe[['Начальный Размер', 'Конечный Размер']] = multiplicity_rost_boxe[~multiplicity_rost_boxe['Размер'].str.contains('/')]['Размер'].str.split('-', expand=True)
<ipython-input-52-c1458ae7d2b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiplicity_rost_boxe[['Начальный Размер', 'Конечный Размер']] = multiplicity_rost_boxe[~multiplicity_rost_boxe['Размер'].str.contains('/')]['Размер'].str.split('-', exp

In [53]:
# Следующая часть используется при разделении артикула по размерам
'''
multiplicity_other_box = sorted_ouer_stock[sorted_ouer_stock['Способ доставки'] != 'Ростовой короб']
multiplicity_other_box['Кол-во пар для общего учета'] = pd.to_numeric(multiplicity_other_box['Свободно'])
ouer_stock_new = pd.concat([multiplicity_other_box, multiplicity_rost_boxe])

ouer_stock_new = ouer_stock_new.replace('None', 0)

ouer_stock_new['Для поиска'] = np.where(
    (ouer_stock_new['Размер'].str.contains('\*')) | (ouer_stock_new['Размер'] == 0),
    ouer_stock_new['Артикул'],
    ouer_stock_new['Артикул'].astype(str) + "-" + ouer_stock_new['Размер'].astype(str)
)

ouer_stock_new = ouer_stock_new.groupby(['Для поиска']).agg({'Кол-во пар для общего учета': 'sum'}).reset_index().fillna(0)

merge_df = pd.merge(merge_df, ouer_stock_new[['Для поиска', 'Кол-во пар для общего учета']], left_on='Артикул', right_on='Для поиска', how='left')
merge_df.rename(columns={'Кол-во пар для общего учета': 'Остатки ЦС, шт'}, inplace=True)
merge_df.drop(columns=['Для поиска'], inplace=True)
'''

<ipython-input-53-de19adca7c07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiplicity_other_box['Кол-во пар для общего учета'] = pd.to_numeric(multiplicity_other_box['Свободно'])


In [106]:
merge_df = pd.merge(merge_df, sorted_ouer_stock[['Артикул', 'Свободно']].groupby('Артикул').sum().reset_index(), left_on='Артикул 1С',
                    right_on='Артикул', how='left')
merge_df.drop(columns=['Артикул'], inplace=True)
merge_df.rename(columns={'Свободно': 'Остатки ЦС, шт'}, inplace=True)
merge_df.fillna(0, inplace=True)

In [107]:
day_start_report = datetime.datetime(2023, 8, 1)
day_finish_report = datetime.datetime(2023, 11, 20)

days_on_sale['Итог дней в наличии'] = np.where(
    pd.to_datetime(day_start_report) >= days_on_sale['На Озоне с'],
    (day_finish_report - pd.to_datetime(day_start_report)).days,
    (day_finish_report - days_on_sale['На Озоне с']).dt.days
)

merge_df = pd.merge(merge_df, days_on_sale.groupby('Артикул 1С').max().reset_index()[['Итог дней в наличии', 'Артикул 1С']], on='Артикул 1С', how='left')

In [108]:
merge_df.fillna(0, inplace=True)

merge_df['Скорость продаж'] = np.where(
    merge_df['Итог дней в наличии'] != 0,
    merge_df['Итог продаж'] / merge_df['Итог дней в наличии'],
    0
)

merge_df['Итог остатков, шт (нужно распродать)'] = merge_df['Остатки ЦС, шт'] * 0.35 + merge_df['Остатки OZON, шт']
merge_df['Оборачиваемость'] = np.where(
    merge_df['Скорость продаж'] > 0,
    merge_df['Итог остатков, шт (нужно распродать)'] / merge_df['Скорость продаж'],
    9999
)

In [56]:
# Поднимаю вверх
'''
merge_df = pd.merge(merge_df, initial_df[['Артикул', 'SKU']].groupby('Артикул').agg({'SKU': 'first'}).reset_index(), on='Артикул', how='left')
merge_df.rename(columns={'SKU': 'Артикул OZON'}, inplace=True)

vectorized_get_hyperlink_ozon = np.vectorize(lambda column: f"https://www.ozon.ru/product/181297357{int(column)}")
merge_df['Ссылка OZON'] = vectorized_get_hyperlink_ozon(merge_df['Артикул OZON'])
'''

In [60]:
len(merge_df)

3121

In [109]:
merge_df['sellThru'] = np.where(
    merge_df['Итог продаж'] * 2 / (merge_df['Итог остатков, шт (нужно распродать)'] + merge_df['Итог продаж']) > 1,
    1,
    np.where(
        merge_df['Итог продаж'] * 2 / (merge_df['Итог остатков, шт (нужно распродать)'] + merge_df['Итог продаж']) <= 0,
        0,
        merge_df['Итог продаж'] * 2 / (merge_df['Итог остатков, шт (нужно распродать)'] + merge_df['Итог продаж'])
    )
)

In [110]:
merge_df = pd.merge(merge_df, ouer_price.groupby('Номенклатура.Артикул').max().reset_index()[['Номенклатура.Артикул', 'Оптовая РУБ']],
                    how='left', left_on='Артикул 1С', right_on='Номенклатура.Артикул')

merge_df = pd.merge(merge_df, ouer_sales_report[['КОЛЛЕКЦИЯ!!!', 'Артикул 1С']], on='Артикул 1С', how='left')

merge_df['РРЦ'] = merge_df['Оптовая РУБ'] * 2
merge_df['ПЦ'] = merge_df['РРЦ'] * 1.5

merge_df.rename(columns={'КОЛЛЕКЦИЯ!!!': 'Коллекция', 'Текущая розн. цена (до скидки)': 'ПЦ'}, inplace=True)
merge_df.drop(columns=['Оптовая РУБ', 'Номенклатура.Артикул'], inplace=True)

merge_df.fillna(0, inplace=True)
merge_df['Совокупная скидка от РРЦ'] =np.where(
    merge_df['Итог продаж'] * merge_df['РРЦ'] != 0,
    1 - (merge_df['Итог выручки'] / (merge_df['Итог продаж'] * merge_df['РРЦ'])),
    0
)

merge_df['Актуальная скидка от РРЦ'] = np.where(
    merge_df['РРЦ'] != 0,
    1 - (merge_df['Актуальная цена'] / merge_df['РРЦ']),
    0
)

<ipython-input-110-2d5962a9f274>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  merge_df = pd.merge(merge_df, ouer_price.groupby('Номенклатура.Артикул').max().reset_index()[['Номенклатура.Артикул', 'Оптовая РУБ']],


In [111]:
len(merge_df)

3277

In [112]:
merge_df['Статус'] = np.where(
    0.9 < merge_df["sellThru"],
    np.where(
        merge_df["Совокупная скидка от РРЦ"] < 0.05,
        "A1",
        np.where(
            (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
            "A2",
            np.where(
                (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                "A3",
                np.where(
                    0.4 <= merge_df["Совокупная скидка от РРЦ"],
                    "A4",
                    "Error"
                )
            )
        )
    ),
    np.where(
        (0.8 < merge_df["sellThru"]) & (merge_df["sellThru"] <= 0.9),
        np.where(
            merge_df["Совокупная скидка от РРЦ"] < 0.05,
            "B1",
            np.where(
                (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
                "B2",
                np.where(
                    (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                    "B3",
                    np.where(
                        (0.4 <= merge_df["Совокупная скидка от РРЦ"]),
                        "B4",
                        "Error"
                    )
                )
            )
        ),
        np.where(
            (0.6 < merge_df["sellThru"]) & (merge_df["sellThru"] <= 0.8),
            np.where(
                merge_df["Совокупная скидка от РРЦ"] < 0.05,
                "C1",
                np.where(
                    (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
                    "C2",
                    np.where(
                        (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                        "C3",
                        np.where(
                            0.4 <= merge_df["Совокупная скидка от РРЦ"],
                            "C4",
                            "Error"
                        )
                    )
                )
            ),
            np.where(
                merge_df["sellThru"] <= 0.6,
                np.where(
                    merge_df["Совокупная скидка от РРЦ"] < 0.05,
                    "D1",
                    np.where(
                        (0.05 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.2),
                        "D2",
                        np.where(
                            (0.2 <= merge_df["Совокупная скидка от РРЦ"]) & (merge_df["Совокупная скидка от РРЦ"] < 0.4),
                            "D3",
                            np.where(
                                0.4 <= merge_df["Совокупная скидка от РРЦ"],
                                "D4",
                                "Error"
                            )
                        )
                    )
                ),
                "Error"
            )
        )
    )
)

In [113]:
merge_df['Новая цена'] = 1

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'A1',
    merge_df['Актуальная цена'] * 1.05,
      np.where(
        merge_df['Статус'] == 'A2',
        merge_df['Актуальная цена'] * 1.05,
        np.where(
            merge_df['Статус'] == 'A3',
            merge_df['Актуальная цена'] * 1.1,
            np.where(
                merge_df['Статус'] == 'A4',
                merge_df['Актуальная цена'] * 1.1,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'B1',
    merge_df['Актуальная цена'],
      np.where(
        merge_df['Статус'] == 'B2',
        merge_df['Актуальная цена'],
        np.where(
            merge_df['Статус'] == 'B3',
            merge_df['Актуальная цена'],
            np.where(
                merge_df['Статус'] == 'B4',
                merge_df['Актуальная цена'] * 1.05,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'C1',
    merge_df['Актуальная цена'] * 0.85,
      np.where(
        merge_df['Статус'] == 'C2',
        merge_df['РРЦ'] * 0.7,
        np.where(
            merge_df['Статус'] == 'C3',
            merge_df['РРЦ'] * 0.6,
            np.where(
                merge_df['Статус'] == 'C4',
                merge_df['РРЦ'] * 0.5,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Статус'] == 'D1',
    merge_df['РРЦ'] * 0.8,
      np.where(
        merge_df['Статус'] == 'D2',
        merge_df['РРЦ'] * 0.7,
        np.where(
            merge_df['Статус'] == 'D3',
            merge_df['РРЦ'] * 0.6,
            np.where(
                merge_df['Статус'] == 'D4',
                merge_df['РРЦ'] * 0.5,
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Новая цена'] = np.where(
    merge_df['Итог дней в наличии'] > 14,
    merge_df['Новая цена'],
    merge_df['Актуальная цена']
)

In [114]:
merge_df['Новинка'] = np.where(
    merge_df['Итог дней в наличии'] > 14,
    'Нет',
    'Да'
)

merge_df['Новая скидка от ПЦ'] = np.where(
    merge_df['ПЦ'] == 0,
    0,
    1 - (merge_df['Новая цена'] / merge_df['ПЦ'])
)

In [115]:
merge_df['Ценообразование'] = 1

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'A1',
    'Поднимаем цены на 5% к текущей цене',
    np.where(
        merge_df['Статус'] == 'A2',
        'Поднимаем цены на 5% к текущей цене',
        np.where(
            merge_df['Статус'] == 'A3',
            'Поднимаем цены на 10% к текущей цене',
            np.where(
                merge_df['Статус'] == 'A4',
                'Поднимаем цены на 10% к текущей цене',
                merge_df['Новая цена']
            )
        )
    )
)

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'B1',
    'Цену не меняем',
    np.where(
        merge_df['Статус'] == 'B2',
        'Цену не меняем',
        np.where(
            merge_df['Статус'] == 'B3',
            'Цену не меняем',
            np.where(
                merge_df['Статус'] == 'B4',
                'Поднимаем цены на 5% к текущей цене',
                merge_df['Ценообразование']
            )
        )
    )
)

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'C1',
    'Снижаем цену на 15%',
    np.where(
        merge_df['Статус'] == 'C2',
        'Скидка 30%',
        np.where(
            merge_df['Статус'] == 'C3',
            'Скидка 40%',
            np.where(
                merge_df['Статус'] == 'C4',
                'Скидка 50%',
                merge_df['Ценообразование']
            )
        )
    )
)

merge_df['Ценообразование'] = np.where(
    merge_df['Статус'] == 'D1',
    'Скидка 20%',
    np.where(
        merge_df['Статус'] == 'D2',
        'Скидка 30%',
        np.where(
            merge_df['Статус'] == 'D3',
            'Скидка 40%',
            np.where(
                merge_df['Статус'] == 'D4',
                'Скидка 50%',
                merge_df['Ценообразование']
            )
        )
    )
)

In [116]:
# Для артикулов с размерами
# merge_df = merge_df.iloc[:, [0, 7, 18, 21, 8, 10, 11, 19, 5, 6, 9, 22, 23, 12, 13, 14, 24, 25, 16, 15, 17, 20, 28, 26, 30, 27, 29]]
# Разделение по артикулам 1С
merge_df = merge_df.iloc[:, [0, 4, 5, 14, 7, 1, 2, 9, 6, 8, 15, 16, 3, 11, 17, 18, 10, 12, 13, 21, 19, 23, 20, 22]]


In [89]:
# pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.4 MB/s eta 0:00:00


In [123]:
test_price = pd.concat([pd.DataFrame({'Артикул': ozon_stock['Артикул'].unique()}),
                  pd.DataFrame({'Артикул': initial_df['Артикул'].unique()})])
test_price = pd.DataFrame({'Артикул': test_price['Артикул'].unique()})

def find_article(row):
    columns = row['Артикул']
    try:
        first_way = columns[:-3]
        second_way = columns[:-6]
    except:
        return pd.Series([0, 0], index=['Артикул 1С', 'Размер'])

    test = ouer_sales_report[ouer_sales_report['Артикул 1С'] == columns]
    if not test.empty:
        size = columns.replace(test['Артикул 1С'].iloc[0], '')
        if len(size) == 0:
            size = 0
        return pd.Series([test['Артикул 1С'].iloc[0], size], index=['Артикул 1С', 'Размер'])

    test = ouer_sales_report[ouer_sales_report['Артикул 1С'] == first_way]
    if not test.empty:
        size = columns.replace(test['Артикул 1С'].iloc[0], '')[1:]
        if len(size) == 0:
            size = 0
        return pd.Series([test['Артикул 1С'].iloc[0], size], index=['Артикул 1С', 'Размер'])

    test = ouer_sales_report[ouer_sales_report['Артикул 1С'] == second_way]
    if not test.empty:
        size = columns.replace(test['Артикул 1С'].iloc[0], '')[1:]
        if len(size) == 0:
            size = 0
        return pd.Series([test['Артикул 1С'].iloc[0], size], index=['Артикул 1С', 'Размер'])

# Apply the function row-wise using axis=1
test_price[['Артикул 1С', 'Размер']] = test_price.apply(find_article, axis=1)

In [124]:
with pd.ExcelWriter('Контроль продаж OZON 24.11.xlsx', engine='xlsxwriter') as writer:
    merge_df.to_excel(writer, sheet_name='Расчет', index=False)
    test_price.to_excel(writer, sheet_name='Цена', index=False)
